# Omdena  Nairobi Challenge  December 2022
**Monitoring Change in Urban Green Areas and Tree Cover using Satellite Imagery (Nairobi County)**

**This notebook was developed for the  [OmdenaAI 2022 Nairobi Challenge](https://omdena.com/chapter-challenges/monitoring-change-in-urban-green-areas-and-tree-cover/).**

**Author: [Hillary Koros](https://github.com/HillaryKoros)**

## Useful links
- [Google Earth Engine](https://earthengine.google.com)
- [geemap.org](https://geemap.org)
- [Google Earth Engine and geemap Python Tutorials](https://www.youtube.com/playlist?list=PLAxJ4-o7ZoPccOFv1dCwvGI6TYnirRTg3) (55 videos with a total length of 15 hours)

## Prerequisite
- A Google Earth Engine account. Sign up [here](https://earthengine.google.com) if needed. 
- [Miniconda](https://docs.conda.io/en/latest/miniconda.html) or [Anaconda](https://www.anaconda.com/products/individual)


## Set up a conda environment

```
conda create -n PyGIS python=3.9
conda activate PyGIS
conda install geemap -c conda-forge
conda install jupyter_contrib_nbextensions -c conda-forge
jupyter contrib nbextension install --user
```

# Problem  Understanding
The main goal of this project is to develop and AI solution that will extract data from satellite imageries hosted on cloud-based platforms (e.g., the Earth Engine’s public data catalog) within a  defined city boundaries(Nairobi), generate statistics on two urban indicators related to environmental sustainability. 
These indicators are:

##  Change in green Areas per Capita
- as defined in the Global Urban Monitoring Framework (UMF-47). The methodology involves the estimation of a city area under vegetation cover for several time periods e.g., the years 2000, 2010, and 2020; the indicator has 2 key metrics: ```change in green areas over time``` and ```change in per capita green areas over time```.

##  Change in Tree Cover
- as defined in the Global Urban Monitoring Framework (UMF-48). The methodology involves estimation of the city area under tree cover for several time periods e.g., the years 2000, 2010, and 2020, and analyzing the change over time.

# Analytical Approach: 
This is a Supervised machine learning problem where we will use  The `Classifier` package which  handles supervised classification by traditional ML algorithms running in Earth Engine. These classifier we will use is RandomForest.

Source: https://developers.google.com/earth-engine/classification


**The general workflow for the project is:**

1. Collect training data. Assemble features which have a property that stores the known class label and properties storing numeric values for the predictors.
2. Instantiate a classifier. Set its parameters if necessary.
3. Train the classifier using the training data.
4. Classify an image or feature collection.
5. Generate statistics and timeseries data 

# Data Requirements:

We  use data would need a collection of Training data,Nairobi County Shapefiles and satellite Imagery for the area and Period under study.

**The Training Data  will Have 4 class Labels**
1. Water.
2. Vegetation.
3. Built-Up.
4. BareLand.


- Nairobi County Bounds, Kenya which I have extracted from: (https://africaopendata.org/dataset/kenya-counties-shapefile) 
- Training Data which is  created using the Drawing Tools in the gee code editor:(https://code.earthengine.google.com/)
- Satellite imagery Landsat  and Sentinel extracted from: ( https://developers.google.com/earth-engine/datasets)

## Step-by-step tutorial

### Import libraries

In [3]:
import ee
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

### Create an interactive map

In [4]:
Map = geemap.Map(center=(-1.3003590,36.816372), zoom=10)
Map

Map(center=[-1.300359, 36.816372], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox…

In [5]:
AOI_shp = './data/Nairobi.shp'
Nai = geemap.shp_to_ee(AOI_shp)
roi = ee.FeatureCollection(Nai)
Map.addLayer(roi, {}, 'Nairobi',False)

Map

Map(bottom=132319.0, center=[-1.300359, 36.816372], controls=(WidgetControl(options=['position', 'transparent_…

### Add data to the map

In [6]:
point = ee.Geometry.Point([36.816372,-1.3003590])

image = (
    ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
    .filterBounds(point)
    .filterDate('2020-01-01', '2020-12-31')
    .sort('CLOUD_COVER')
    .first()
    .select('B[1-7]')
    .clip(roi)
)

vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.centerObject(point, 8)
Map.addLayer(image, vis_params, "Landsat-8")
Map

Map(bottom=132319.0, center=[-1.300359, 36.816371999999994], controls=(WidgetControl(options=['position', 'tra…

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
ndvi = image.normalizedDifference(['B5', 'B4']).rename('NDVI')
#image.addBands(ndvi)
withNDVI= image.addBands(ndvi)
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
Map.addLayer(ndvi, ndviParams, 'NDVI image')
Map

Map(bottom=264196.0, center=[-1.2029145493452746, 36.73862457275391], controls=(WidgetControl(options=['positi…

### Check image properties

In [22]:
ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()

'2020-02-20'

In [23]:
image.get('CLOUD_COVER').getInfo()

1.3

In [35]:
withNDVI.get('system:band_names').getInfo()

['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'NDVI']

In [25]:
file_path = os.path.abspath('file.geojson')
with open(file_path) as f:
    json_data = json.load(f)
    

ee_data = geojson_to_ee(json_data)
Map.addLayer(ee_data, {}, "Training",False)
Map

Map(bottom=264196.0, center=[-1.2029145493452746, 36.73862457275391], controls=(WidgetControl(options=['positi…

In [26]:
print(ee_data.size().getInfo())

192


In [27]:
print(ee_data.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[36.84045664162497, -1.310484563835185], [36.85255876870016, -1.310484563835185], [36.85255876870016, -1.3007024055691276], [36.84045664162497, -1.3007024055691276], [36.84045664162497, -1.310484563835185]]]}, 'id': '1_1_1_0', 'properties': {'LC': 2}}


### Train the classifier

In [36]:
# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI']


# This property of the table stores the land cover labels.
label = 'LC'

# Overlay the points on the imagery to get training.
trainingD = withNDVI.select(bands).sampleRegions(
    **{'collection': ee_data, 'properties': [label], 'scale': 10}
)

#split into training and validation
# Adds a column of deterministic pseudorandom numbers.
sample = trainingD.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

# print(training.first().getInfo())

# print(validation.first().getInfo())
# Train a RandomForest classifier with 50 trees default parameters.
trained = ee.Classifier.smileRandomForest(50).train(training, label, bands)

In [37]:
# print(training.first().getInfo())
print(validation.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '1_1_1_0_0', 'properties': {'B2': 910, 'B3': 1112, 'B4': 1279, 'B5': 1879, 'B6': 2629, 'B7': 2475, 'LC': 2, 'NDVI': 0.18999366462230682, 'random': 0.878334566787231}}


### Classify the image

In [38]:
# Classify the image with the same bands used for training.
result = withNDVI.select(bands).classify(trained)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classified')
Map

Map(bottom=264196.0, center=[-1.2029145493452746, 36.73862457275391], controls=(WidgetControl(options=['positi…

<IPython.core.display.Javascript object>

In [50]:
# Area Calculation for Images
vegetation = result.eq(1)
Map.addLayer(vegetation, {'min':0, 'max':1, 'palette': ['white', 'green']}, 'Green Cover')
Map

Map(bottom=132491.0, center=[-1.5365284406473707, 37.067184448242195], controls=(WidgetControl(options=['posit…

In [59]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
filename = os.path.join(out_dir, '2020NDVI.tif')

In [61]:
NDVI_image =withNDVI.select('NDVI') # select the 5th image in a collection
geemap.ee_export_image(
    NDVI_image, filename=filename, scale=30, region=roi.geometry())

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Koros\Downloads\2020NDVI.tif
